In [1]:
import dotenv, os, time, random, re, csv, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

percorso_credenziali = '../credenziali/credenziali_cartella.env'
dotenv.load_dotenv(override = True, dotenv_path = percorso_credenziali)

cartella = os.getenv('Download')
url = "https://www.hp.com/us-en/shop/cv/select-country-and-language"

larghezza_finestra = 1366
lunghezza_finestra = 768

attesa_pagina = 30
attesa_elemento = 3

# rimuovi_paesi = {
    # "South Africa",
    # "Switzerland in French",
    # "Switzerland in German"
# }
# sostituisci_paesi = {
    # "Switzerland in English": "Switzerland"
# }

paesi_scelti = {
    "Germany",
    "France",
    "Spain"#,
    # "United Kingdom"
}

indici_menu = {
    0: 'Laptops',
    1: 'Desktops',
    3: 'Monitors'
}
indici_submenu = {
    1: 'Home',
    2: 'Business'#,
#   3: 'Gaming'
}

In [2]:
personalizza = Options()
personalizza.add_argument("--lang=en-GB")
personalizza.add_experimental_option("excludeSwitches", ["enable-logging"])
personalizza.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)
# personalizza.add_argument("--headless=new")
# personalizza.add_argument("--disable-gpu")
# personalizza.add_argument(f"--window-size={larghezza_finestra},{lunghezza_finestra}")

percorso_chrome = ChromeDriverManager().install()
servizio = Service(percorso_chrome)

paesi = {}
elementi = []
dati_scraping = []

driver = webdriver.Chrome(service=servizio, options=personalizza)
driver.set_page_load_timeout(attesa_pagina)
driver.implicitly_wait(attesa_elemento)
wait = WebDriverWait(driver, 10)
driver.maximize_window()

driver.get(url)

wait.until(expected_conditions.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click()
wait.until(expected_conditions.element_to_be_clickable((By.ID, "view_all_stores"))).click()

wait.until(expected_conditions.element_to_be_clickable((By.ID, "emeaList")))
filtro = driver.find_element(By.ID, "emeaList")
elementi = filtro.find_elements(By.CSS_SELECTOR, "h2.countrywrap")

for elemento in elementi:
    elemento = elemento.find_element(By.CSS_SELECTOR, "a")
    paese = elemento.text.strip()
    paese = paese[8:]
    sito = elemento.get_attribute("href")
    paesi[paese] = sito

# paesi = {
    # sostituisci_paesi.get(paese, paese): sito
    # for paese, sito in paesi.items()
    # if paese not in rimuovi_paesi
# }

paesi = {
    paese: sito
    for paese, sito in paesi.items()
    if paese in paesi_scelti
}

finestra_principale = driver.current_window_handle

for paese, sito in paesi.items():
    lista_finestre = driver.window_handles
    
    wait.until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, f'a[href="{sito}"]'))).click()
    
    wait.until(lambda attesa_nuova_finestra: len(attesa_nuova_finestra.window_handles) > len(lista_finestre))
    finestra_paese = [finestra for finestra in driver.window_handles if finestra not in lista_finestre][0]

    driver.switch_to.window(finestra_paese)
    wait.until(lambda attesa_caricamento_pagina: attesa_caricamento_pagina.execute_script("return document.readyState") == "complete")
    
    links = driver.find_elements(By.CSS_SELECTOR, 'li[class^="group/navigation"]')
    formattazione_pagina = 'try'
    if links == []:
        links = driver.find_elements(By.CSS_SELECTOR, 'div[class^="main-menu__link-container"]')
        formattazione_pagina = 'except'
    
    links = [links[numero_indice] for numero_indice in indici_menu.keys()]
    
    for link, categoria in zip (links, indici_menu.values()):
        ActionChains(driver).move_to_element(link).perform()
        
        pagine_scelte = []
        
        time.sleep(random.uniform(2,4))
        try:
            pagine_scelte = link.find_elements(By.CSS_SELECTOR, 'a[data-gtm-id^="L2"]')
            pagine_scelte = [pagine_scelte[numero_indice] for numero_indice in indici_submenu.keys()]
        except:
            pagine_scelte = link.find_elements(By.CSS_SELECTOR, 'a.sub-menu__link')
            pagine_scelte = [pagine_scelte[numero_indice] for numero_indice in indici_submenu.keys()]
        
        for pagina, ambito in zip (pagine_scelte, indici_submenu.values()):
            dati_scraping.append({
                'Paese': paese,
                'Categoria': categoria,
                'Ambito': ambito,
                'Link': pagina.get_attribute("href")
            })
    
    driver.close()
    driver.switch_to.window(finestra_principale)

driver.quit()

In [3]:
dati = []

for dato in dati_scraping:
    paese = dato['Paese']
    categoria = dato['Categoria']
    ambito = dato['Ambito']
    url = dato['Link']
    
    driver = webdriver.Chrome(service=servizio, options=personalizza)
    driver.set_page_load_timeout(attesa_pagina)
    driver.implicitly_wait(attesa_elemento)
    wait = WebDriverWait(driver, 10)
    driver.maximize_window()
    
    driver.get(url)
    wait.until(expected_conditions.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click()
    wait.until(lambda attesa_caricamento_pagina: attesa_caricamento_pagina.execute_script("return document.readyState") == "complete")
    
    impaginazione = driver.find_elements(By.CSS_SELECTOR, 'button[class^="ProductsPerPageOptions_button"][tabindex="0"]')
    impaginazione = [bottone for bottone in impaginazione if not re.search(r'\d', bottone.text)]
    
    if impaginazione != []:
        impaginazione = impaginazione[0]
        impaginazione.click()
        wait.until(lambda attesa_caricamento_pagina: attesa_caricamento_pagina.execute_script("return document.readyState") == "complete")

    time.sleep(random.uniform(2,4))
    wait.until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'div[class^="HorizontalProductTile_container"]')))
    pagina = driver.page_source
    analisi = BeautifulSoup(pagina, 'html.parser')
    
    prodotti = analisi.select('div[class^="HorizontalProductTile_container"]')
    
    for prodotto in prodotti:
        titolo = prodotto.select_one('p[class*="Typography-module_titleS"]')
        titolo = titolo.get_text(strip=True) if titolo else None
        sku = prodotto.select_one('div[class*="HorizontalProductTile_skuWrapper"]')
        prezzo = prodotto.select_one('div[class^="Typography-module_root"][class*="Typography-module_boldL"]')
        prezzo = prezzo.get_text(strip=True) if prezzo else None
        if sku:
            sku = sku.get_text(strip=True)
            sku = re.search(r'([A-Za-z0-9]{7})#AB', sku)
            if sku:
                sku = sku.group(1)
        
        dati.append({
            'Paese': paese,
            'Categoria': categoria,
            'Ambito': ambito,
            'Titolo': titolo,
            'Sku': sku,
            'Prezzo': prezzo
        })
    
    driver.quit()
    time.sleep(random.uniform(5,10))

In [4]:
file = f'{cartella}\\capstonehp.csv'

In [5]:
with open(file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "Paese",
        "Categoria",
        "Ambito",
        "Titolo",
        "Sku",
        "Prezzo"
    ])
    writer.writeheader()
    writer.writerows(dati)

In [6]:
dataset = pd.read_csv(file)
dataset

,Paese,Categoria,Ambito,Titolo,Sku,Prezzo
0,Germany,Laptops,Home,HP OmniBook 5 16-bf0754ng,C39C1EA,"799,00 €"
1,Germany,Laptops,Home,HP OmniBook 5 14-he0775ng,C78RZEA,"1 049,00 €"
2,Germany,Laptops,Home,HP OmniBook 7 17-dc0795n - NVIDIA® GeForce RTX...,BM9U8EA,"1 899,00 €"
3,Germany,Laptops,Home,HP Laptop - 17-cp3775ng (2023),7N1C2EA,"829,00 €"
4,Germany,Laptops,Home,OMEN Transcend Laptop 16-u1776ng (2023),9P1U0EA,"2 599,00 €"
...,...,...,...,...,...,...
976,Spain,Monitors,Business,"Monitor FHD USB-C HP Pro de la serie 5 de 23,8...",9D9V7AA,"252,89€"
977,Spain,Monitors,Business,Monitor FHD HP de la serie 3 Pro de 27 pulgada...,B0CG8UT,"180,29€"
978,Spain,Monitors,Business,"Monitor FHD HP de la serie 3 Pro de 23,8 pulga...",B0BU9UT,"156,09€"
979,Spain,Monitors,Business,HP Serie 3 Pro 322ph FHD,B0BN7UT,"143,99€"


In [7]:
dataset['Prezzo'] = (
    dataset['Prezzo']
    .replace({'€': '', ' ': ''}, regex=True)
    .str.replace(',', '.', regex=False)
)

dataset['Prezzo'] = pd.to_numeric(dataset['Prezzo'], errors='coerce')

dataset = dataset[dataset['Sku'] != '']

dataset.to_csv(file, index=False)
dataset

,Paese,Categoria,Ambito,Titolo,Sku,Prezzo
0,Germany,Laptops,Home,HP OmniBook 5 16-bf0754ng,C39C1EA,799.00
1,Germany,Laptops,Home,HP OmniBook 5 14-he0775ng,C78RZEA,1049.00
2,Germany,Laptops,Home,HP OmniBook 7 17-dc0795n - NVIDIA® GeForce RTX...,BM9U8EA,1899.00
3,Germany,Laptops,Home,HP Laptop - 17-cp3775ng (2023),7N1C2EA,829.00
4,Germany,Laptops,Home,OMEN Transcend Laptop 16-u1776ng (2023),9P1U0EA,2599.00
...,...,...,...,...,...,...
976,Spain,Monitors,Business,"Monitor FHD USB-C HP Pro de la serie 5 de 23,8...",9D9V7AA,252.89
977,Spain,Monitors,Business,Monitor FHD HP de la serie 3 Pro de 27 pulgada...,B0CG8UT,180.29
978,Spain,Monitors,Business,"Monitor FHD HP de la serie 3 Pro de 23,8 pulga...",B0BU9UT,156.09
979,Spain,Monitors,Business,HP Serie 3 Pro 322ph FHD,B0BN7UT,143.99
